In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersCohn
from genomic_benchmarks.models.torch_cnn import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config 


## Config

In [3]:
config = {
    "use_padding": False,
    "run_on_gpu": True,
    "dataset": HumanEnhancersCohn,
    "number_of_classes": 2,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
#   vocabulary that is not present in the training set but is present in the test set
    "vocab_to_add": [],
}
check_config(config)

config is correct


## Choose the dataset

In [4]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Unzipping...Done.


## Tokenizer and vocab

In [5]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 6
{'<eos>': 5, 'G': 4, 'A': 3, 'C': 2, 'T': 1, '<bos>': 0}


## Dataloader and batch preparation

In [6]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  500


## Model

In [7]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

## Training

In [8]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0
Train metrics: 
 Accuracy: 63.8%, Avg loss: 0.654706 

Epoch 1
Train metrics: 
 Accuracy: 63.5%, Avg loss: 0.654620 

Epoch 2
Train metrics: 
 Accuracy: 65.6%, Avg loss: 0.648896 

Epoch 3
Train metrics: 
 Accuracy: 65.1%, Avg loss: 0.648712 

Epoch 4
Train metrics: 
 Accuracy: 67.2%, Avg loss: 0.639706 

Epoch 5
Train metrics: 
 Accuracy: 68.3%, Avg loss: 0.636274 

Epoch 6
Train metrics: 
 Accuracy: 68.7%, Avg loss: 0.633181 

Epoch 7
Train metrics: 
 Accuracy: 71.3%, Avg loss: 0.630310 

Epoch 8
Train metrics: 
 Accuracy: 69.9%, Avg loss: 0.630840 

Epoch 9
Train metrics: 
 Accuracy: 69.0%, Avg loss: 0.628594 

Epoch 10
Train metrics: 
 Accuracy: 71.4%, Avg loss: 0.626259 

Epoch 11
Train metrics: 
 Accuracy: 69.7%, Avg loss: 0.626749 

Epoch 12
Train metrics: 
 Accuracy: 71.3%, Avg loss: 0.623227 

Epoch 13
Train metrics: 
 Accuracy: 71.9%, Avg loss: 0.621211 

Epoch 14
Train metrics: 
 Accuracy: 72.9%, Avg loss: 0.619314 



## Testing

In [9]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  140.57966035604477
num_batches 218
correct 4723
size 6948
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.644861 

